In [1]:
# YOLO for 2D luggage box detection

In [35]:
from ultralytics import YOLO
import cv2

# Load a model
model = YOLO("yolo11s.pt")  # load an official model
# model = YOLO("path/to/best.pt")  # load a custom model
img_path = '3.jpg'
# Predict with the model
results = model(img_path, conf = 0.1)  # predict on an image
# 提取框的坐标
boxes = results[0].boxes.xyxy.cpu().numpy()  # 提取 xyxy 坐标


# 如果有框，保留面积最大的框
if len(boxes) > 0:
    # 计算每个框的面积：面积 = (x2 - x1) * (y2 - y1)
    areas = [(box[2] - box[0]) * (box[3] - box[1]) for box in boxes]
    max_idx = areas.index(max(areas))  # 找到最大面积的索引
    largest_box = boxes[max_idx]  # 最大的框

    # 加载原始图像
    img = cv2.imread(img_path)
    
    # 绘制最大框
    x1, y1, x2, y2 = map(int, largest_box)
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)  # 绿色框，线宽2

    # 显示图像
    cv2.imshow("Largest Box", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    # 保存图像
    cv2.imwrite("box.jpg" + img_path, img)

else:
    print("未检测到任何框！")


image 1/1 c:\Users\Administrator\Desktop\3DLuggageDetetcion\3DluggageDetect\3.jpg: 480x640 1 suitcase, 188.2ms
Speed: 2.0ms preprocess, 188.2ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)


In [20]:
print(results[0].boxes)

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([28.])
conf: tensor([0.3613])
data: tensor([[ 82.8514,  76.1576, 242.2824, 150.8749,   0.3613,  28.0000]])
id: None
is_track: False
orig_shape: (192, 256)
shape: torch.Size([1, 6])
xywh: tensor([[162.5669, 113.5162, 159.4311,  74.7173]])
xywhn: tensor([[0.6350, 0.5912, 0.6228, 0.3892]])
xyxy: tensor([[ 82.8514,  76.1576, 242.2824, 150.8749]])
xyxyn: tensor([[0.3236, 0.3967, 0.9464, 0.7858]])


In [1]:
from ultralytics import YOLO
import cv2

# 加载模型
model = YOLO("yolo11s.pt")

# 预测图像
results = model("IDS_CAM7_6B_POS1__VI__1569846775.jpg")  # 替换为你的图像路径



In [ ]:
# 提取检测框
boxes = results[0].boxes  # 获取第一个图像的检测框
img = cv2.imread("IDS_CAM7_6B_POS1__VI__1569846775.jpg")  # 加载原始图像

# 筛选行李类别（假设类别编号为 28，需根据模型类别设置）
target_class = 28  # 替换为实际的行李类别编号
for box in boxes:
    # 提取框坐标和类别
    x1, y1, x2, y2 = map(int, box.xyxy[0])  # 左上角 (x1, y1) 和右下角 (x2, y2)
    cls = int(box.cls[0])  # 类别编号
    conf = box.conf[0]  # 置信度

    # 如果类别是行李，绘制框
    if cls == target_class:
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)  # 绘制绿色矩形框
        label = f"Luggage {conf:.2f}"
        cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# 保存或显示结果
cv2.imshow("Luggage Detection", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [1]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

# 加载 YOLOv8 模型
def load_yolo_model(weights_path):
    model = YOLO(weights_path)
    return model

# 进行检测并绘制框
def detect_and_draw(image_path, model, save_path=None):
    # 加载图片
    results = model(image_path, conf=0.5)  # 设置置信度阈值
    annotated_image = results[0].plot()  # 获取绘制框的图片
    
    # 显示图片
    plt.imshow(cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

    # 保存图片
    if save_path:
        cv2.imwrite(save_path, annotated_image)
        print(f"Saved annotated image to {save_path}")

# 主流程
def main():
    weights_path = "yolov11n.pt"  # 替换为你的权重文件路径
    image_path = "IDS_CAM7_6B_POS1__VI__1569846775.jpg"  # 替换为测试图片路径
    save_path = "result.jpg"  # 保存结果的路径（可选）

    # 加载模型
    model = load_yolo_model(weights_path)

    # 检测并绘制
    detect_and_draw(image_path, model, save_path)

if __name__ == "__main__":
    main()


In [ ]:
import torch
import torchvision.transforms as T
from PIL import Image
import cv2
import matplotlib.pyplot as plt

# 加载模型
def load_model(weights_path):
    # 假设使用的是 torchvision 提供的 Faster R-CNN
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
    num_classes = 2  # 修改为你模型的类别数（例如背景 + 行李 = 2）
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
    model.load_state_dict(torch.load(weights_path))
    model.eval()  # 切换为评估模式
    return model

# 加载图片并预处理
def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    transform = T.Compose([
        T.ToTensor(),
    ])
    return transform(image).unsqueeze(0)  # 增加batch维度

# 在图片上绘制框
def draw_boxes(image_path, predictions, threshold=0.5):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    for box, score in zip(predictions['boxes'], predictions['scores']):
        if score > threshold:
            x1, y1, x2, y2 = box.int().numpy()
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(image, f"{score:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    plt.imshow(image)
    plt.axis("off")
    plt.show()

# 主流程
def main():
    weights_path = "path/to/your/weights.pth"  # 替换为你的权重文件路径
    image_path = "path/to/your/image.jpg"  # 替换为测试图片路径

    # 加载模型和图片
    model = load_model(weights_path)
    image_tensor = load_image(image_path)

    # 进行预测
    with torch.no_grad():
        predictions = model(image_tensor)[0]

    # 绘制预测结果
    draw_boxes(image_path, predictions)

if __name__ == "__main__":
    main()


In [1]:
from ultralytics import YOLO
import cv2
# Load a model
model = YOLO("yolo11n.pt")  # load an official model
# model = YOLO("path/to/best.pt")  # load a custom model

# Predict with the model
results = model("https://ultralytics.com/images/boats.jpg")  # predict on an image
res = results[0].plot()
cv2.imshow("res", res)
cv2.waitKey(0)

In [1]:
from ultralytics import YOLO